# TravelAssist AI

## Part 1: Introduction

#### Project Background

In today's busy and fast paced life, everyone wants a quick escape every now and then in form of a holiday. However, the overwhelming number of choices and the lack of personalized assistance can make the holiday planning daunting. To address this, we have developed **TravelAssist AI, a chatbot that combines the power of large language models and rule-based functions to ensure accurate and reliable information delivery**.


#### Problem Statement

*Given a dataset containing information about holiday packages (Package name, destination, No. of days, Sighseeing etc.), build a chatbot that parses the dataset and provides accurate holiday recommendations based on user requirements*.


#### Dataset
Data is gathered form Kaggle's make my trip dataset on holiday package from the follwing link: 'https://www.kaggle.com/datasets/promptcloud/travel-listing-from-makemytrip' 

Original dataset contained 30k rows, however for this problem statement this has been trimmed down to just 4 rows.






#### Approach:

1. **Conversation and Information Gathering**: The chatbot will utilize language models to understand and generate natural responses. Through a conversational flow, it will ask relevant questions to gather information about the user's requirements.
2. **Information Extraction**: Once the essential information is collected, rule-based functions come into play, extracting relevant holiday packages that best matches the user's needs.
3. **Personalized Recommendation**: Leveraging this extracted information, the chatbot engages in further dialogue with the user, efficiently addressing their queries and aiding them in answering any question related to the package.

## Part 2: System Design

#### Dataset

We have a dataset `HolidayData.xlsx` where  each row describes the features of a holiday package. The chatbot that we build will leverage LLMs to converse with user to identify their specific needs and then recommend the matching holiday packages from the dataset

#### Building the Chatbot

Now let's go ahead and understand the system design for the chatbot.

[Chatbot_sys_design.png](https://drive.google.com/uc?id=1j-mw_dNcbxGcelQ0PmkDB0nKOpauU1wX)

As shown in the image, the chatbot contains the following layers:
- Intent Clarity Layer
- Intent Confirmation Layer
- Product Mapping Layer
- Product Information Extraction Layer
- Product Recommendation Layer

##### Major functions behind the Chatbot

Let's now look at a brief overview of the major functions that form the chatbot. We'll take a deep dive later



- `initialize_conversation()`: This initializes the variable conversation with the system message.
- `get_chat_model_completions()`: This takes the ongoing conversation as the input and returns the response by the assistant
- `moderation_check()`: This checks if the user's or the assistant's message is inappropriate. If any of these is inappropriate, it ends the conversation.
- `intent_confirmation_layer()`: This function takes the assistant's response and evaluates if the chatbot has captured the user's profile clearly. Specifically, this checks if the following properties for the user has been captured or not 'Destination','Package','Origin','Duration','Budget'
- `dictionary_present()`: This function checks if the final understanding of user's profile is returned by the chatbot as a python dictionary or not. If there is a dictionary, it extracts the information as a Python dictionary.
- `compare_holiday_with_user()`: This function compares the user's requirements with the different holiday packages and come back with the top 3 recommendations.
- `initialize_conv_reco()`: Initializes the recommendations conversation

## Part 3: Implementation

### 3.1 - Import the libraries

Let's start by importing the libraries that we'll require for this project. Following are the ones:
- openai
- pandas
- os, json, ast



In [504]:
import os,json,ast
import openai
import pandas as pd

In [505]:
# read the API key
openai.api_key = open("OPENAI_API_Key.txt", "r").read().strip()

### 3.2 - Implementing Intent Clarity and Intent Confirmation Layers

Let's start with the first part of the implementation - building the `intent clarity` and `intent confirmation` layers. As mentioned earlier, this layer helps in identifying the user requirements and passing it on to the product matching layer. Here are the functions that we would be using for building these layers:

- `initialize_conversation()`: This initializes the variable conversation with the system message. used prompt engineering and chain of thought reasoning, the function will enable the chatbot to keep asking questions until the user requirements have been captured in a dictionary. It also includes Few Shot Prompting(sample conversation between the user and assistant) to align the model about user and assistant responses at each step.


In [507]:
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"
    example_user_req = {'Destination': 'Manali', 'Package': 'Deluxe', 'Origin': 'New Delhi', 'Duration': '4', 'Budget': '15000 INR'}

    system_message = f"""

    You are an intelligent holiday planner and your goal is to find the best holiday package for a user.
    Your objective is to extract the correct values for the following keys in a dictionary:
    - 'Destination'
    - 'Package'
    - 'Origin'
    - 'Duration'
    - 'Budget'
    
    The values must be inferred **only from the user's responses**. Do not assume or assign values randomly.

    {delimiter} **Duration Extraction Rules:** {delimiter}
    - The user may specify duration in different ways:
      - **"6 nights"** → Extract **6**  
      - **"for 6 nights"** → Extract **6**  
      - **"I want to stay 6 nights"** → Extract **6**  
      - **"I am planning for 6 nights"** → Extract **6**  
      - **"I plan to stay for 5 days"** → Extract **5**  
    - Once the duration is successfully extracted, **do not ask again**.  
    - If unclear, ask **"How many nights do you plan to stay?"**  
    - If the response is vague (e.g., "a few days"), ask for clarification.  
    - If the user has already provided a clear response (e.g., **"I would like to plan for 6 nights"**), store it and **move to the next question.**    
    - If the response is vague (e.g., "a few days"), ask for clarification.

    {delimiter} **Important Rules for Extracting Values:** {delimiter}
    - **Budget:** Extract a numerical value. The minimum budget should be **6500 INR**. If the user provides a lower budget, inform them that no packages are available in that range.
    - **Duration:** Extract a numerical value from user input (e.g., "3 nights," "for 5 days").  
      - If a user provides **"3 nights"**, consider it as `3`.  
      - If a user provides **"for 5 days"**, consider it as `5`.  
      - If unclear, politely **ask the user again in a new way** (instead of repeating the same question).
    - **Origin:** Must be either **"Mumbai"** or **"New Delhi"**. If the user mentions another city, inform them that only Mumbai and New Delhi are supported.
    - **Package:** Offer the user **four choices** – Premium, Deluxe, Luxury, Standard.

    {delimiter} **Improvement: Avoid Unnecessary Modifications Prompt** {delimiter}
    - Once all details are collected, **DO NOT ask "Is there anything else you would like to modify?"**
    - Instead, immediately show available package options.

    {delimiter} **Structured Thought Process for Filling Values** {delimiter}
    **Thought 1:** Start by confirming the **Destination** and **Duration**.  
      - If the user mentions a destination but **not the duration**, ask about duration in a natural, conversational way.  
      - Example:  
        - User: "I want to visit Manali."  
        - Assistant: "Great! How many nights do you plan to stay?"  
      - Ensure that a **valid numerical value** is extracted for duration.

    **Thought 2:** Ask for missing details (**Package, Origin, and Budget**) logically.  
      - Do not ask multiple questions at once.  
      - Example:  
        - User: "I'll start from Mumbai."  
        - Assistant: "Thank you! Could you let me know your preferred package – Premium, Deluxe, Luxury, or Standard?"  

    **Thought 3:** Confirm that all details have been collected correctly. If any value is missing, ask for clarification.  

    {delimiter} **Example Conversation Flow:** {delimiter}
    **User:** "Hi, I want to visit Manali."  
    **Assistant:** "Great! How many nights do you plan to stay in Manali?"  
    **User:** "I want to stay for 3 nights."  
    **Assistant:** "Thank you! Which package would you like to choose – Premium, Deluxe, Luxury, or Standard?"  
    **User:** "I'll take Deluxe."  
    **Assistant:** "Thanks! Could you confirm whether your starting city is Mumbai or New Delhi?"  
    **User:** "New Delhi."  
    **Assistant:** "Great! Lastly, please share your per-person budget for the trip."  
    **User:** "My budget is 15,000 INR."  
    **Assistant:** {example_user_req}  

    Start the conversation with a **friendly welcome message** and guide the user to share their preferences.
    """
    conversation = [{"role": "system", "content": system_message}]
    return conversation


In [508]:
debug_conversation = initialize_conversation()
print(debug_conversation)

[{'role': 'system', 'content': '\n\n    You are an intelligent holiday planner and your goal is to find the best holiday package for a user.\n    Your objective is to extract the correct values for the following keys in a dictionary:\n    - \'Destination\'\n    - \'Package\'\n    - \'Origin\'\n    - \'Duration\'\n    - \'Budget\'\n    \n    The values must be inferred **only from the user\'s responses**. Do not assume or assign values randomly.\n\n    #### **Duration Extraction Rules:** ####\n    - The user may specify duration in different ways:\n      - **"6 nights"** → Extract **6**  \n      - **"for 6 nights"** → Extract **6**  \n      - **"I want to stay 6 nights"** → Extract **6**  \n      - **"I am planning for 6 nights"** → Extract **6**  \n      - **"I plan to stay for 5 days"** → Extract **5**  \n    - Once the duration is successfully extracted, **do not ask again**.  \n    - If unclear, ask **"How many nights do you plan to stay?"**  \n    - If the response is vague (e.g., 

Building the next function
- `get_chat_model_completions()`: This takes the ongoing conversation as the input and returns the response by the assistant

In [509]:
import os, json, ast
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [510]:
# Define a Chat Completions API call
def get_chat_model_completions(input, json_format = False):
    MODEL = 'gpt-3.5-turbo'

    system_message_json_output = """<<. Return output in JSON format to the key output.>>"""

    # If the output is required to be in JSON format
    if json_format == True:
        # Append the input prompt to include JSON response as specified by OpenAI
        input[0]['content'] += system_message_json_output

        # JSON return type specified
        chat_completion_json = openai.chat.completions.create(
            model = MODEL,
            messages = input,
            response_format = { "type": "json_object"},
            seed = 1234)

        output = json.loads(chat_completion_json.choices[0].message.content)

    # No JSON return type specified
    else:
        chat_completion = openai.chat.completions.create(
            model = MODEL,
            messages = input,
            seed = 2345)

        output = chat_completion.choices[0].message.content

    return output

Let's pass the initialized conversation `debug_conversation` and see what is the assistant's response.

In [512]:
debug_introduction = get_chat_model_completions(debug_conversation)
print(debug_introduction)

Welcome! I can help you find the perfect holiday package. Where would you like to go?


We can play around a bit and add the following user's input `debug_user_input` to the conversation `debug_conversation` and see what the assistant responds with.

In [513]:
debug_user_input = "Hi, I am Uday Kiran Pujaari. I want to visit Kashmir."

In [514]:
debug_conversation.append({"role": "user", "content": debug_user_input})
debug_response_assistant = get_chat_model_completions(debug_conversation)
print(debug_response_assistant)

Great! How many nights do you plan to stay in Kashmir?


In [515]:
debug_user_input = "i would like to visit for 5 nights"

In [516]:
debug_conversation.append({"role": "user", "content": debug_user_input})
debug_response_assistant = get_chat_model_completions(debug_conversation)
print(debug_response_assistant)

Great choice! Which package would you like to choose for your trip to Kashmir – Premium, Deluxe, Luxury, or Standard?


In [517]:
debug_user_input = "i'd go with Premium"

In [518]:
debug_conversation.append({"role": "user", "content": debug_user_input})
debug_response_assistant = get_chat_model_completions(debug_conversation)
print(debug_response_assistant)

Thank you for providing the information.  
Could you please confirm whether your starting city is Mumbai or New Delhi?


In [519]:
debug_user_input = "Mumbai"

In [520]:
debug_conversation.append({"role": "user", "content": debug_user_input})
debug_response_assistant = get_chat_model_completions(debug_conversation)
print(debug_response_assistant)

Great choice, Uday Kiran Pujaari! Could you please share your per-person budget for the trip to Kashmir?


In [521]:
debug_user_input = 'my max budget is 50000 INR'

In [522]:
debug_conversation.append({"role": "user", "content": debug_user_input})
debug_response_assistant = get_chat_model_completions(debug_conversation)
print(debug_response_assistant)

Great choice! Here is the holiday package option I have tailored for you:

- Destination: Kashmir
- Duration: 5 nights
- Package: Premium
- Origin: Mumbai
- Budget: 50000 INR

Enjoy your trip to Kashmir! If you need any further assistance or want to explore more options, feel free to ask.


Typically, whenever the chatbot is interacting with the user, all the conversations should be moderated to identify any inappropriate content. Let's look at the function that can help with it.

- `moderation_check()`: This checks if the user's or the assistant's message is inappropriate. If any of these is inappropriate, one can add a break statement to end the conversation.

In [523]:
# Define a function called moderation_check that takes user_input as a parameter.

def moderation_check(user_input):
    # Call the OpenAI API to perform moderation on the user's input.
    response = openai.moderations.create(input=user_input)

    # Extract the moderation result from the API response.
    moderation_output = response.results[0].flagged
    # Check if the input was flagged by the moderation system.
    if response.results[0].flagged == True:
        # If flagged, return "Flagged"
        return "Flagged"
    else:
        # If not flagged, return "Not Flagged"
        return "Not Flagged"

Let's test moderation on the `debug_user_input`

In [524]:
debug_moderation = moderation_check(debug_user_input)
print(debug_moderation)

Not Flagged


Let's now test moderation on some other text.

Let's now check moderation on the assistant's response `debug_response_assistant`.

In [525]:
moderation_check(debug_response_assistant)

'Not Flagged'

- `intent_confirmation_layer()`: This function takes the assistant's response and evaluates if the chatbot has captured the user's profile clearly. Specifically, this checks if the following properties for the user has been captured or not
   - Destination
   - Package
   - Origin
   - Duration
   - Budget

In [526]:
def intent_confirmation_layer(response_assistant):
    delimiter = "####"
    prompt = f"""
    You are a senior evaluator who has an eye for detail.
    You are provided an input. You need to evaluate if the input has the following keys: 'Destination','Package','Origin','Duration','Budget'
    Next you need to evaluate if the keys have the the values filled correctly.
    - The value for the key 'Budget' needs to contain a number with currency.
    - The value of key 'Package' needs to be either  'Premium', 'Deluxe' , 'Luxury' or 'Standard'
    - The value of key 'Duration' needs to contain a number
    - The value of key 'Origin' should either be 'New Delhi' or 'Mumbai'
    Only output a one-word string in JSON format at the key 'result' - Yes/No.
    Thought 1 - Output a string 'Yes' if the values are correctly filled for all keys, otherwise output 'No'.
    Thought 2 - If the answer is No, mention the reason in the key 'reason'.
    Thought 3 - Think carefully before the answering..
    """

    messages=[{"role": "system", "content":prompt },
              {"role": "user", "content":f"""Here is the input: {response_assistant}""" }]

    response = openai.chat.completions.create(
                                    model="gpt-3.5-turbo",
                                    messages = messages,
                                    response_format={ "type": "json_object" },
                                    seed = 1234
                                    # n = 5
                                    )

    json_output = json.loads(response.choices[0].message.content)

    return json_output

Let's apply the function to the assistant's reponse and see if it has captured the user profile.

In [527]:
debug_confirmation = intent_confirmation_layer(debug_response_assistant)
print(debug_confirmation)

{'result': 'Yes'}


Now, you can keep adding user and assistant responses to debug_conversation and get to a point where intent_confirmation_layer() 


- `dictionary_present()`: This function checks if the final understanding of user's profile is returned by the chatbot is a Python dictionary or not. This is important as it'll be used later on for finding the right holiday packages using dictionary matching.

In [528]:
def dictionary_present(response):
    delimiter = "####"
    user_req = {'Destination': 'Katra','Package': 'Premium','Origin': 'New Delhi','Duration': '5','Budget': '15000 INR'}
    prompt = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {user_req}.
            Your task is to just extract and return only the python dictionary from the input.
            The output should match the format as {user_req}.
            The output should contain the exact keys and values as present in the input.

            Here are some sample input output pairs for better understanding:
            {delimiter}
            input 1: - Destination: Manali - Package: Luxury - Origin: Delhi - Duration: 4 - Budget: 50,000 INR
            output 1: {{'Destination': 'Manali', 'Package': 'Luxury', 'Origin': 'New Delhi', 'Duration': '4', 'Budget': '50000'}}

            input 2: {{'Destination':     'jaipur', 'Package':     'Standard', 'Origin':    'Mumbai', 'Duration': '4','Budget': '90,000'}}
            output 2: {{'Destination': 'Jaipur', 'Package': 'Standard', 'Origin': 'Mumbai', 'Duration': '4', 'Budget': '90000'}}

            input 3: Here is your user profile 'Destination': 'Ooty','Package': 'Deluxe','Origin': 'New Delhi','Duration': '5','Budget': '100000 INR'
            output 3: {{'Destination': 'Ooty', 'Package': 'Deluxe', 'Origin': 'New Delhi', 'Duration': '5', 'Budget': '100000'}}
            {delimiter}

            """

    messages = [{"role": "system", "content":prompt },
                {"role": "user", "content":f"""Here is the user input: {response}""" }]
    
    confirmation = get_chat_model_completions(messages, json_format = True)

    return confirmation.get("output", confirmation) 

Let's start by passing the `debug_response_assistant`.

In [529]:
response_dict_n = dictionary_present(debug_response_assistant)
print(response_dict_n)

{'Destination': 'Kashmir', 'Duration': '5 nights', 'Package': 'Premium', 'Origin': 'Mumbai', 'Budget': '50000'}


What if you pass something like this where it is not in the form of a dictionary? Or some key or some values are missing? Let's see.

In [530]:
debug_response_assistant_n = f"""Thank you for providing your budget. Based on your budget of 50,000 INR, I will consider this while recommending suitable package options for you.
Here is the final recommendation for your travel:

- Destination: Katra
- Package: Premium
- Origin: Mumbai
- Duration: 5
- Budget: 50,000 INR

Please note that these specifications are based on your requirements within your budget. Let me know if there's anything else I can assist you with!"""

In [531]:
response_dict_n = dictionary_present(debug_response_assistant_n)
print(response_dict_n)

{'Destination': 'Katra', 'Package': 'Premium', 'Origin': 'Mumbai', 'Duration': '5', 'Budget': '50000 INR'}


### 3.3 Implementing the Product Mapping and Information Extraction Layers

In this section, we take in the output of the previous layers, i.e. the user requirements, which is in the format of a Python dictionary, and extract the relevant holiday recommendations based on that. Here are the functions that we will use to help us implement the information extraction and product matching layers


- `product_map_layer()`: This function is responsible for extracting key features and criteria from holiday package dataset. Here's a breakdown of how it works:
    - Uses a prompt that assign it the role of a holiday expert, whose objective is to extract key features as per requirements
    -  Assign specific rules for each feature.
    - Includes Few Shot Prompting(sample conversation between the user and assistant) to demonstrate the expected result of the       feature extraction.

In [532]:
def product_map_layer(Destination,Package,Origin,Duration):

  delimiter = "#####"
  holiday_spec = {
      "Destination":"(Holiday destination)",
      "Package":"(Type of the holiday package)",
      "Origin":"(Origin city of the holiday)",
      "Duration":"(Maximum duration of the holiday)"
      
  }

 
  prompt=f"""
  You are a Holiday expert whose job is to extract the key features of a holiday package as per their requirements.
  Extract all the features above acccording to following rules: \
  {delimiter}
  Destination: Extract the first word until you find '|'. If there is only one word, extract the same from {Destination}

  Package: Extract the exact value from {Package} 

  Origin: Extract the exact value from {Origin}

  Duration: Add all the numbers you find and output the sum from {Duration}

  {delimiter}

  {delimiter}
  Here are some input output pairs for few-shot learning:
  input1: "Gangtok|Lachung|Gangtok|Darjeeling, Deluxe, New Delhi, 2N Gangtok . 2N Lachung . 1N Gangtok . 2N Darjeeling"
  output1: {{'Destination': 'Gangtok','Package':'Deluxe','Origin':'New Delhi','Duration':'7'}}

  input2: "Cochin|Munnar|Thekkady|Allepey|Kovalam and Poovar, Standard, Mumbai, 1N Cochin . 2N Munnar . 1N Thekkady . 1N Allepey . 2N Kovalam and Poovar"
  output2: {{'Destination': 'Cochin','Package':'Standard','Origin':'Mumbai','Duration':'7'}}

  input3: "Dharamshala, Deluxe, New Delhi, 3N Dharamshala"
  output3: {{'Destination': 'Dharamshala','Package':'Deluxe','Origin':'New Delhi','Duration':'3'}}
  {delimiter}

  Follow the above instructions step-by-step and output only the dictionary {holiday_spec} without any additional text for the following holiday package {Destination,Package,Origin,Duration}.
  """

  messages=[{"role": "user","content":prompt}]

  response = get_chat_model_completions(messages, json_format = True)

  return response

- `extract_dictionary_from_string()`: This function takes in the output of the previous layer and extracts the user requirements dictionary

In [533]:
import ast
import re

def extract_dictionary_from_string(string):
    regex_pattern = r"\{[^{}]+\}"

    if isinstance(string, dict):
        return string
        
    dictionary_matches = re.findall(regex_pattern, string)

    # Extract the first dictionary match and convert it to lowercase
    if dictionary_matches:
        dictionary_string = dictionary_matches[0]
        dictionary_string = dictionary_string.lower()

        # Convert the dictionary string to a dictionary object using ast.literal_eval()
        dictionary = ast.literal_eval(dictionary_string)
    return dictionary

- `compare_holiday_with_user()`: This function compares the user's profile with the different holiday packages and come back with the top  recommendations. It will perform the following steps:
    - It will take the user requirements dictionary as input
    - Filter the holidays based on their per person budget, keeping only the ones within the user's budget.
    - Match the user requirement with holiday data.
    - Return the matching holidays as a JSON-formatted string.

In [534]:
def compare_holiday_with_user(user_req_string):
    holiday_df = pd.read_excel('HolidayData.xlsx')

    ## Create a new column "holiday_feature" that contains the dictionary of the product features
    holiday_df['holiday_feature'] = holiday_df.apply(
        lambda x: product_map_layer(x['Destination'], x['Package Type'], x['Start City'], x['Itinerary']), 
        axis=1
    )

    # Extract user requirements into a dictionary
    user_requirements = extract_dictionary_from_string(user_req_string)

    # Extract and process the budget safely
    budget_str = user_requirements.get('Budget', '0').replace(',', '').split()
    budget = int(budget_str[0]) if budget_str else 0  # Handles missing budget values

    # Filter holidays where 'Per Person Price' is within budget
    filtered_holiday = holiday_df[holiday_df['Per Person Price'] <= budget].copy()

    print('user_requirements:', user_requirements)

    # Safely remove 'Budget' from user_requirements
    user_requirements.pop('Budget', None)

    # Convert holiday packages to dictionaries for comparison
    filtered_holiday['holiday_feature'] = filtered_holiday['holiday_feature'].apply(extract_dictionary_from_string)

    # Compare user requirements with holiday packages
    mask = filtered_holiday['holiday_feature'].apply(
        lambda x: all(user_requirements.get(key) == x.get(key) for key in user_requirements)
    )
    
    result_df = filtered_holiday[mask]

    return result_df.to_json(orient='records')

Now that you have the `compare_holiday_with_user()` function ready, let's pass the `response_dict_n` to the function to get relevant recommendation.

In [535]:
recommended_holiday = compare_holiday_with_user(response_dict_n)
print(recommended_holiday)

user_requirements: {'Destination': 'Katra', 'Package': 'Premium', 'Origin': 'Mumbai', 'Duration': '5', 'Budget': '50000 INR'}
[]


### 3.4: Product Recommendation Layer

Finally, we come to the product recommendation layer. It takes the output from the `compare_holiday_with_user` function in the previous layer and provides the recommendations to the user. It has the following steps.
1. Initialize the conversation for recommendation.
2. Generate the recommendations and display in a presentable format.
3. Ask questions basis the recommendations.



In [536]:
def initialize_conv_reco(products):
    system_message = f"""
    You are an intelligent holiday expert and you are tasked with the objective to \
    solve the user queries about any product from the catalogue: {products}.\
    You should keep the user requirements in mind while answering the questions.\

    Start with a brief summary of each holiday in the following format, in decreasing order of price per person:
    1. <Holiday Name> : <Major attractions of the holiday>, <Price per person in Rs>
    2. <Holiday Name> : <Major attractions of the holiday>, <Price per person in Rs>

    """
    conversation = [{"role": "system", "content": system_message }]
    return conversation

Let's initialize the conversation for recommendation.

In [537]:
debug_conversation_reco = initialize_conv_reco(recommended_holiday)

Let's see what the assistant responds with the new initialization.

In [538]:
debug_recommendation = get_chat_model_completions(debug_conversation_reco)
print(debug_recommendation + '\n')

I'm sorry, but it seems that there are no holidays in the catalogue to provide information about. If you have any other questions or need assistance with something else, feel free to ask!



Now, you can converse with the chatbot on the filtered products.

In [539]:
debug_conversation_reco.append({"role": "assistant", "content": debug_recommendation})

In [540]:
debug_user_input = "Will it cover some adventure activities?"

In [541]:
debug_conversation_reco.append({"role": "user", "content": debug_user_input})
debug_response_asst_reco = get_chat_model_completions(debug_conversation_reco)
print('\n' + debug_response_asst_reco + '\n')


Since there are no specific holidays in the catalogue to refer to, I don't have information on adventure activities included in any particular holiday at the moment. However, if you have a destination in mind or specific activities you are interested in, I can help suggest some adventurous holiday options for you. Just let me know your preferences!



You can repeat the process of appending the assistant and user messages and test the chatbot out.

### 3.5 Dialogue Management System

Bringing everything together, we create a `diagloue_mgmt_system()` function that contains the logic of how the different layers would interact with each other. This will be the function that we'll call to initiate the chatbot

In [542]:
def dialogue_mgmt_system():
    conversation = initialize_conversation()
    
    introduction = get_chat_model_completions(conversation)
    print(introduction + '\n')
    recommended_holiday = None
    user_input = ''

    while user_input.lower() != "exit":
        user_input = input("")

        moderation = moderation_check(user_input)
        if moderation == 'Flagged':
            print("Sorry, this message has been flagged. Please restart your conversation.")
            break

        if recommended_holiday is None:
            conversation.append({"role": "user", "content": user_input})

            response_assistant = get_chat_model_completions(conversation)

            moderation = moderation_check(response_assistant)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            confirmation = intent_confirmation_layer(response_assistant)
            print("Intent Confirmation Yes/No:", confirmation.get('result'))

            if "No" in confirmation:
                conversation.append({"role": "assistant", "content": response_assistant})
                print("\n")
                print(response_assistant)
            else:
                print("\n")
                print(response_assistant)
                
                # Extract user details from response
                response = dictionary_present(response_assistant)

                if not response or not isinstance(response, dict):  # Check if extraction failed
                    continue

                required_keys = {'Destination', 'Package', 'Origin', 'Duration', 'Budget'}
                missing_keys = required_keys - set(response.keys())

                if missing_keys:
                    print(f"\nIt looks like some details are missing: {', '.join(missing_keys)}.")
                    print("Can you please provide the missing details?")
                    continue  # Ask user for details again

                print("\nThank you for providing all the information. Kindly wait, while I fetch the products: \n")

                # Safe deletion of Budget key
                response.pop('Budget', None)

                # Compare user requirements with available holidays
                recommended_holiday = compare_holiday_with_user(response)

                conversation_reco = initialize_conv_reco(recommended_holiday)
                conversation_reco.append({"role": "user", "content": "This is my user tour details: " + str(response)})
                recommendation = get_chat_model_completions(conversation_reco)

                moderation = moderation_check(recommendation)
                if moderation == 'Flagged':
                    print("Sorry, this message has been flagged. Please restart your conversation.")
                    break

                conversation_reco.append({"role": "assistant", "content": str(recommendation)})
                print(str(recommendation) + '\n')

        else:
            conversation_reco.append({"role": "user", "content": user_input})

            response_asst_reco = get_chat_model_completions(conversation_reco)

            moderation = moderation_check(response_asst_reco)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            print('\n' + response_asst_reco + '\n')
            conversation.append({"role": "assistant", "content": response_asst_reco})


In [543]:
dialogue_mgmt_system()

Welcome! I can help you find the perfect holiday package. Where would you like to go?



 I would like to visit goa


Intent Confirmation Yes/No: No


Great choice! How many nights do you plan to stay in Goa?


 I plan to stay for 6 nights


Intent Confirmation Yes/No: No


Great! Which package would you like to choose – Premium, Deluxe, Luxury, or Standard?


...


Here are some luxury holiday options for your trip to Goa for 6 nights starting from New Delhi:

1. Exclusive Seaside Villa Experience: Stay in a luxurious seaside villa, relax on private beaches, indulge in gourmet dining, and enjoy personalized services. Price per person: Rs 60,000
2. Private Yacht Party: Experience the ultimate luxury with a private yacht party, exclusive dining experiences, and VIP services in Goa. Price per person: Rs 45,000

These luxury holiday options offer a premium experience for your trip to Goa. Let me know if you need more details or have any other preferences!



 I would like to change the number of nights to 3 nights



Here are some luxury holiday options in Goa for 3 nights starting from New Delhi:

1. Luxury Beachfront Villa Experience: Indulge in a lavish stay at a beachfront villa, with private beach access, personalized services, and stunning views. Price per person: Rs 50,000
2. Exclusive Spa Retreat: Relax and rejuvenate at a luxury spa resort, enjoy world-class spa treatments, gourmet dining, and serene surroundings. Price per person: Rs 35,000

These holidays offer a luxurious experience and are perfect for a short yet indulgent getaway. Let me know if you need more information or have any other preferences!



 exit



If you have any more questions in the future or need further assistance, feel free to ask. Have a great day!

